# Identifica produtos contidos em cada venda

Leitura do DataFrame de vendas

In [1]:
import pandas as pd

In [2]:
vendas_df = pd.read_csv('../data/gold/2025_11_final.csv')

In [3]:
vendas_df

,id_venda,data_venda,status_venda,unidades_vendidas,receita_por_produto,tarifas_impostos_venda,receita_envio,tarifa_envio,custo_envio,diff_custo_envio,...,variacao_anuncio,preco_unitario_anuncio,tipo_anuncio,reclamacao_encerrada,count_produto,cor_kimono,tamanho_kimono,STR/BZ,AKAI/FEM,FX
0,2000009795156777,31 de outubro de 2025,A caminho,1,"329,99","-37,95","0,0","-24,95","-24,95","0,0",...,Preto|A3,"329,99",Clássico,0,True,Preto,A3,1,0,1
1,2000009795033373,31 de outubro de 2025,Entregue,1,"329,99","-37,95","0,0","-24,95","-24,95","0,0",...,Preto|A3,"329,99",Clássico,0,True,Preto,A3,1,0,1
2,2000013624370268,31 de outubro de 2025,Mediação em espera de resposta do Mercado Livre,1,"329,99","-37,95","36,99","-61,94","-61,94","0,0",...,Preto|A2,"329,99",Clássico,0,True,Preto,A2,1,0,1
3,2000009794910165,31 de outubro de 2025,Entregue,1,"355,3","-40,86","0,0","-23,45","-23,45","0,0",...,Preto|A1,"355,3",Clássico,0,True,Preto,A1,0,1,1
4,2000009794919421,31 de outubro de 2025,Entregue,1,"329,99","-37,95","0,0","-24,95","-24,95","0,0",...,Preto|A2,"329,99",Clássico,0,True,Preto,A2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,2000013234018272,1 de outubro de 2025,Cancelada pelo comprador,1,"329,99","-37,95","0,0","-24,95","-24,95","0,0",...,Preto|A1,"329,99",Clássico,0,False,Preto,A1,0,0,0
1330,2000013233981392,1 de outubro de 2025,Mediação finalizada. Te demos o dinheiro.,1,"329,99","-37,95","13,99","-37,44","-37,44","0,0",...,Preto|A1,"329,99",Clássico,1,True,Preto,A1,1,0,1
1331,2000013233610400,1 de outubro de 2025,Entregue,1,"366,29","-42,12","0,0","-23,45","-23,45","0,0",...,Preto|A1,"366,29",Clássico,0,True,Preto,A1,0,1,1
1332,2000009408783295,1 de outubro de 2025,Entregue,1,"329,99","-37,95","0,0","-26,95","-26,95","0,0",...,Preto|A3,"329,99",Clássico,0,True,Preto,A3,1,0,1
